In [1]:
def read_file(filepath):
    with open(filepath) as f:
        str_text = f.read()
    return str_text

In [2]:
# read_file('moby_dick_four_chapters.txt')

In [3]:
# read_file('melville-moby_dick.txt')

In [4]:
import spacy

In [5]:
nlp = spacy.load('en_core_web_sm',disable=['parser', 'tagger','ner'])

In [6]:
nlp.max_length = 1198623

In [7]:
def separate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [8]:
d = read_file('moby_dick_four_chapters.txt')

In [9]:
tokens = separate_punc(d)

In [10]:
len(tokens)

11394

In [11]:
# 25 words ---> network predict #26

In [12]:
train_len = 25 + 1
text_sequences = []
for i in range(train_len, len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)

In [13]:
type(text_sequences)

list

In [14]:
' '.join(text_sequences[0])

'call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on'

In [15]:
' '.join(text_sequences[1])

'me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore'

In [16]:
' '.join(text_sequences[2])

'ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore i'

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer

C:\Users\SSAFY\anaconda3\envs\nlp_course\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\SSAFY\anaconda3\envs\nlp_course\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\SSAFY\anaconda3\envs\nlp_course\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\SSAFY\anaconda3\envs\nlp_course\lib\site-packages

In [18]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)

In [19]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [20]:
sequences[1]

[14,
 265,
 51,
 263,
 416,
 87,
 222,
 129,
 111,
 962,
 262,
 50,
 43,
 37,
 321,
 7,
 23,
 555,
 3,
 150,
 261,
 6,
 2704,
 14,
 24,
 965]

In [21]:
for i in sequences[0]:
    print(f"{i}: {tokenizer.index_word[i]}")

964: call
14: me
265: ishmael
51: some
263: years
416: ago
87: never
222: mind
129: how
111: long
962: precisely
262: having
50: little
43: or
37: no
321: money
7: in
23: my
555: purse
3: and
150: nothing
261: particular
6: to
2704: interest
14: me
24: on


In [22]:
# tokenizer.word_counts

In [23]:
vocabulary_size = len(tokenizer.word_counts)

In [24]:
vocabulary_size

2709

In [25]:
type(sequences)

list

In [26]:
import numpy as np

In [27]:
sequences = np.array(sequences)

In [28]:
sequences

array([[ 964,   14,  265, ..., 2704,   14,   24],
       [  14,  265,   51, ...,   14,   24,  965],
       [ 265,   51,  263, ...,   24,  965,    5],
       ...,
       [ 960,   12,  168, ...,  264,   53,    2],
       [  12,  168, 2703, ...,   53,    2, 2709],
       [ 168, 2703,    3, ...,    2, 2709,   26]])

In [29]:
from tensorflow.keras.utils import to_categorical

In [30]:
X = sequences[:,:-1]

In [31]:
y = sequences[:,-1]

In [32]:
y = to_categorical(y, num_classes=vocabulary_size+1)

In [33]:
seq_len = X.shape[1]

In [34]:
X.shape

(11368, 25)

In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding

In [36]:
def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size, 25, input_length=seq_len))
    model.add(LSTM(150, return_sequences=True))
    model.add(LSTM(150))
    model.add(Dense(150, activation='relu'))

    model.add(Dense(vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   
    model.summary()
    
    return model

In [37]:
model = create_model(vocabulary_size+1, seq_len)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 25)            67750     
_________________________________________________________________
lstm (LSTM)                  (None, 25, 150)           105600    
_________________________________________________________________
lstm_1 (LSTM)                (None, 150)               180600    
_________________________________________________________________
dense (Dense)                (None, 150)               22650     
_________________________________________________________________
dense_1 (Dense)              (None, 2710)              409210    
Total params: 785,810
Trainable params: 785,810
Non-trainable params: 0
_________________________________________________________________


In [38]:
from pickle import dump, load

In [39]:
model.fit(X, y, batch_size=128, epochs=2, verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/2
11368/11368 [==============================] - 7s 601us/sample - loss: 6.8570 - acc: 0.0475
Epoch 2/2
11368/11368 [==============================] - 6s 529us/sample - loss: 6.3868 - acc: 0.0501


In [40]:
model.save('my_mobydick_model.h5')

In [41]:
dump(tokenizer, open('my_simpletokenizer', 'wb'))

In [42]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [59]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    output_text = []
    
    # 25 words
    input_text = seed_text
    
    for i in range(num_gen_words):
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        pred_word_ind = model.predict_classes(pad_encoded, verbose=0)[0]
        pre_word = tokenizer.index_word[pred_word_ind]
        input_text += ' '+pre_word
        output_text.append(pre_word)
    
    return ' '.join(output_text)

In [47]:
text_sequences[0]

['call',
 'me',
 'ishmael',
 'some',
 'years',
 'ago',
 'never',
 'mind',
 'how',
 'long',
 'precisely',
 'having',
 'little',
 'or',
 'no',
 'money',
 'in',
 'my',
 'purse',
 'and',
 'nothing',
 'particular',
 'to',
 'interest',
 'me',
 'on']

In [48]:
import random
random.seed(101)
random_pick = random.randint(0, len(text_sequences))

In [49]:
random_seed_text = text_sequences[random_pick]

In [50]:
random_seed_text

['and',
 'throwing',
 'the',
 'clothes',
 'to',
 'one',
 'side',
 'he',
 'really',
 'did',
 'this',
 'in',
 'not',
 'only',
 'a',
 'civil',
 'but',
 'a',
 'really',
 'kind',
 'and',
 'charitable',
 'way',
 'i',
 'stood',
 'looking']

In [51]:
seed_text = ' '.join(random_seed_text)

In [52]:
seed_text

'and throwing the clothes to one side he really did this in not only a civil but a really kind and charitable way i stood looking'

In [60]:
generate_text(model, tokenizer, seq_len, seed_text=seed_text, num_gen_words=25)

'the the the the the the the the the the the the the the the the the the the the the the the the the'

In [62]:
from tensorflow.keras.models import load_model

In [63]:
model = load_model('epochBIG.h5')

In [64]:
tokenizer = load(open('epochBIG', 'rb'))

In [65]:
generate_text(model, tokenizer, seq_len, seed_text=seed_text, num_gen_words=25)

"at that stubb ' my frame roman eyes of his own power for the whale 's grain to wrenched progeny for a fever drawn up"